In [1]:
from keras.models import Sequential, ask_to_proceed_with_overwrite
from keras.layers import Dense
from keras.layers.core import Activation, Dense
from keras.layers import SimpleRNN, LSTM,  GRU,SeparableConvolution2D, Conv2D
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Dense, GlobalAveragePooling2D
import glob
import scipy
import numpy as np
from PIL import Image
import random
import pandas as pd
import keras
from keras.applications import VGG16, VGG19
from keras.models import Model
from keras.layers.core import Activation, Dense
import tensorflow as tf

def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    #vgg_model = VGG16( weights='imagenet', include_top=False )
    vgg_model = VGG16(weights='imagenet', include_top=False)

    for layer in vgg_model.layers[:-3]:
        layer.trainable = False

    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(output_dim ,activation='softmax')(x)

# this is the model we will train
    model = Model(input=vgg_model.input, output=predictions)

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate

    return model


def load_model(weights_path, output_dim):

    model = getModel( output_dim ) 
#     for k,layer in model.layers_by_depth.items()[1:]:
#         layer[0].trainable = False
    
    model.load_weights(weights_path)
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'categorical_crossentropy'])   
    return model

ModuleNotFoundError: No module named 'keras'

In [ ]:
def load_data(N,output_dim):
    X = []
    y = []
    
    
    filenames = sorted(glob.glob("/Users/francescoferrari/Downloads/logo_dataset_6/*/*.jpg"))
    
    junk_filenames = glob.glob("/Users/francescoferrari/Downloads/junk_processed/*/*.jpg")
    
    random.shuffle(junk_filenames)
    
    junk_filenames = junk_filenames[:len(filenames)]
    filenames += junk_filenames
    
    random.shuffle(filenames)
    X_mean = 0.0
    for filename in filenames[:N]:
        label = np.zeros(output_dim,)
        idx = labels_dict.get(filename.split("/")[-2], 6)
        label[idx] = 1.0
        img = np.asarray(Image.open(filename).resize((224, 224), Image.ANTIALIAS))/255.
        if img.shape == (224,224,3):
            X_mean += img.mean()
            y.append(label)
            X.append(img)

        
    limit = int(len(X)/10*8)
    print(len(X))
    print(limit)
    X_mean = X_mean/float(len(X))
    X_train = np.array(X[:limit])
    X_val = np.array(X[limit:])
    y_train = np.array(y[:limit])
    y_val = np.array(y[limit:])   
    X_train -= X_mean
    X_val -= X_mean
    print(X_mean)
    print("completed formatting!")
    return X_train, y_train, X_val, y_val

In [ ]:
labels_dict = {"Cocacola":0, "LOreal":1,"LouisVuitton":2,"Nike":3,"Pepsi":4,"Samsung":5}

In [ ]:
X_train, y_train, X_val, y_val = load_data(100000,7 )

In [ ]:

output_dim = 7
epochs = 100
#model = getModel( output_dim ) 

model = load_model("weights-improvement-CPU-VGG16-LOGO7-02-0.8741.hdf5",7)

In [ ]:
filepath="weights-improvement-CPU-VGG16-LOGO7-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

earlyStopping=EarlyStopping(monitor='val_acc', patience=4, verbose=0, mode='auto')
callbacks_list = [earlyStopping, checkpoint]
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy', 'categorical_crossentropy'])

class_weights = {0:30,1:30,2:30,3:30,4:30,5:30,6:1}

h = model.fit(X_train, y_train, validation_data=(X_val, y_val),class_weight=class_weights,
          batch_size=16, nb_epoch=epochs, verbose=1,callbacks=callbacks_list)

In [ ]:

125741.794427222/3600

In [ ]:
model.summary()

In [ ]:
d = pd.read_csv("top_frames_LOGO7.txt", header=None)

In [ ]:
s = sorted(list(zip(d[0], d[6])), key=lambda k: -k[1])

In [ ]:
s[:2000]